In [1]:
import glob
import re
import pandas as pd
import spacy
import os
import gensim
from gensim.models.word2vec import Word2Vec
import itertools 

/Users/melvinwevers/.pyenv/versions/3.6.5/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/melvinwevers/.pyenv/versions/3.6.5/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/melvinwevers/.pyenv/versions/3.6.5/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/melvinwevers/.pyenv/versions/3.6.5/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)
/Users/melvinwevers/.pyenv/versions/3.6.5/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibi

In [2]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [3]:
nlp = spacy.blank('nl')
nlp.add_pipe(nlp.create_pipe('sentencizer'))

print('Pipeline: ', nlp.pipe_names)

Pipeline:  ['sbd']


/Users/melvinwevers/.pyenv/versions/3.6.5/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/melvinwevers/.pyenv/versions/3.6.5/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)


In [4]:
path = '../../newspapers/vk'
path2 = 'sentences_new'
allFiles = glob.glob(path + "/*.tsv")

In [5]:
def is_noise(token, remove_stopwords = True, remove_punctuation = True):
    token_flags = [token.is_stop, token.is_punct]
    do_flags = [remove_stopwords, remove_punctuation]
    flags = [token and do for token, do in zip(token_flags, do_flags) ]
    return any(flags)

def normalize(token, do_lemmatize = True, do_lower = True ):
    if do_lemmatize and do_lower:
        return token.lemma_.lower()
    if do_lemmatize:
        return token.lemma_
    if do_lower:
        return token.lower_

def extract_sentences(texts, do_lower = True, do_lemmatize= True, remove_stopwords = True, remove_punctuation = True):
    for doc in nlp.pipe(texts, batch_size=10000, n_threads = 20):
        yield [[normalize(token, do_lower, do_lemmatize) for token in sent
                if not is_noise(token, remove_stopwords, remove_punctuation)]
                for sent in doc.sents]

In [74]:
def preprocess_year(start_y, end_y):
    for file_ in allFiles:
        filename = re.sub(path + '/', '', file_)
        filename = filename[12:]
        for year in range(start_y, end_y):       
            if filename.startswith(str(year)):
                print(filename)
                df = pd.read_csv(file_, sep='\t', encoding='utf-8', header=None)
                df = df.sample(frac=0.01, replace=False)
                df.columns = ['date', 'page', 'size', 'min_x', 'min_y', 'max_x', 'max_y', 'w', 'h', 'image_url', 'ocr_url', 'ocr']
                df = df[df['ocr'].notnull()] #remove empty rows
                df['ocr'].apply(extract_sentences)
                out_path = "articles/{0}.txt".format(year)
                with open(out_path, 'a+') as out_file:
                    articles = (article for article in df['ocr'].values)
                    for sentences in extract_sentences(articles, do_lemmatize = False):
                        print(*[' '.join(sentence) for sentence in sentences if sentence],
                              file=out_file,
                              sep=os.linesep)

In [75]:
preprocess_year(1950, 1951)

1950-23507.tsv


In [40]:
def iter_file(path):
    with open(path, 'r') as f:
        for sentence in f:
            if sentence.split():
                yield sentence.lower().split()
                        
class Sentences():
    def __init__(self, data_path):
        self.data_path = data_path
    def __iter__(self):
        return iter_file(self.data_path)

def iter_load_sentences(start_year, end_year, data_path):
    for year in range(start_year, end_year + 1):
            year_path = os.path.join(data_path, str(year) + '.txt')
            yield iter_file(year_path)

            
class TimestampedSentences():
    def __init__(self, start_year, end_year, data_path):
        self.start_year = start_year
        self.end_year = end_year
        self.data_path = data_path
    def __iter__(self):
        return itertools.chain.from_iterable(iter_load_sentences(self.start_year, self.end_year, self.data_path))

In [13]:
def train_embeddings(sentences, num_features = 300,
                     min_word_count = 5, num_workers = 4, context = 4, downsampling = 1e-3, sg=1,
                     hierarchical_softmax=0, negative_sampling_num_words=5):
    model = Word2Vec(iter=1,
            workers=num_workers,
            size=num_features, min_count = min_word_count,
            window = context, sample = downsampling,
            sg=sg, hs=hierarchical_softmax, negative=negative_sampling_num_words)
    bigram_transformer = gensim.models.Phrases(sentences)
    bigram = gensim.models.phrases.Phraser(bigram_transformer)
    corpus = list(bigram[sentences])
    model.build_vocab(corpus)
    model.train(corpus, 
                total_examples=model.corpus_count, 
                epochs=model.iter)
    return model.wv

def save_embeddings(embeddings, path, file):
    save_path = "{0}{1}".format(path, file)
    try:
        embeddings.save_word2vec_format(save_path, binary=True)
        
        #model.wv.save_word2vec_format('model.txt', binary=False)
        
    except FileNotFoundError:
        os.mkdir(path)
        embeddings.save(save_path)

In [78]:
num_features = 300    # Word vector dimensionality                      
min_word_count = 5   # Minimum word count                        
context = 10         # Context window size                                                                                    
downsampling = 10e-5  # Downsample setting for frequent words
num_workers = 6
hierarchical_softmax = 1
skip_gram = 1
negative_sampling_num_words = 0

periods = [#('1985-1989', TimestampedSentences(1985, 1989, '../data/articles')),
           #('1980-1984', TimestampedSentences(1980, 1984, '../data/articles'))
           ('1950-1951', TimestampedSentences(1950, 1951, 'articles'))
          ]

for identifier, sentences in periods:
    embeddings = train_embeddings(sentences, num_features=num_features,
                                  min_word_count=min_word_count, num_workers=num_workers, 
                                  context=context, downsampling=downsampling, sg=skip_gram,
                                  hierarchical_softmax=hierarchical_softmax,
                                  negative_sampling_num_words=negative_sampling_num_words)
    save_embeddings(embeddings, 'embeddings/', "embeddings_{0}".format(identifier))

2018-10-03 15:16:16,292 : INFO : collecting all words and their counts
2018-10-03 15:16:16,294 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2018-10-03 15:16:16,534 : INFO : PROGRESS: at sentence #10000, processed 101129 words and 109474 word types
2018-10-03 15:16:16,582 : INFO : collected 129684 word types from a corpus of 121883 words (unigram + bigrams) and 12170 sentences
2018-10-03 15:16:16,583 : INFO : using 129684 counts as vocab in Phrases<0 vocab, min_count=5, threshold=10.0, max_vocab_size=40000000>
2018-10-03 15:16:16,583 : INFO : source_vocab length 129684
2018-10-03 15:16:17,711 : INFO : Phraser built with 338 338 phrasegrams
2018-10-03 15:16:18,401 : INFO : collecting all words and their counts
2018-10-03 15:16:18,402 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-10-03 15:16:18,431 : INFO : PROGRESS: at sentence #10000, processed 98358 words, keeping 28111 word types
2018-10-03 15:16:18,438 : INFO : collected 3219

In [43]:
from gensim.models.keyedvectors import KeyedVectors
embeddings_60s = KeyedVectors.load('embeddings/embeddings_1960-1969')
embeddings_60s.most_similar('efficiency', topn=10)

2018-10-03 14:47:45,255 : INFO : loading Word2VecKeyedVectors object from embeddings/embeddings_1960-1969
2018-10-03 14:47:46,024 : INFO : loading vectors from embeddings/embeddings_1960-1969.vectors.npy with mmap=None
2018-10-03 14:47:46,114 : INFO : setting ignored attribute vectors_norm to None
2018-10-03 14:47:46,115 : INFO : loaded embeddings/embeddings_1960-1969
2018-10-03 14:47:46,117 : INFO : precomputing L2-norms of word weight vectors


[('rentabiliteit', 0.8244878053665161),
 ('kostenbesparingen', 0.8007539510726929),
 ('specialisatie', 0.7978460788726807),
 ('structurele', 0.7843245267868042),
 ('bedrijfsvoering', 0.7802438139915466),
 ('mechanisatie', 0.7753682136535645),
 ('bedrijfsbeleid', 0.7747997045516968),
 ('premieverhoging', 0.7741789221763611),
 ('textielindustrie', 0.7692541480064392),
 ('verslechtering', 0.7676308155059814)]

In [5]:
from gensim.models.keyedvectors import KeyedVectors
m_0 = KeyedVectors.load_word2vec_format('embeddings/vecs_1945-55_v2-2.bin', binary=True, unicode_errors='ignore')
#m_1 = KeyedVectors.load_word2vec_format('embeddings/vecs_1965-75.bin', binary=True, unicode_errors='ignore')

#wv_from_bin = KeyedVectors.load_word2vec_format(datapath("euclidean_vectors.bin"), binary=True)  # C binary format

2018-10-08 11:34:47,421 : INFO : loading projection weights from embeddings/vecs_1945-55_v2-2.bin
2018-10-08 11:34:47,428 : WARNING : duplicate word 'n' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:47,443 : WARNING : duplicate word 'n' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:47,457 : WARNING : duplicate word '2' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:47,466 : WARNING : duplicate word 'f' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:47,469 : WARNING : duplicate word '3' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:47,477 : WARNING : duplicate word '4' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:47,478 : WARNING : duplicate word 'be' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:47,486 : WARNING : duplicate word 'ver' in embeddings/vecs_1945-55_v2-2.bin, ig

2018-10-08 11:34:47,829 : WARNING : duplicate word 'pro' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:47,831 : WARNING : duplicate word 'ht' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:47,832 : WARNING : duplicate word 'ht' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:47,835 : WARNING : duplicate word 'kun' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:47,839 : WARNING : duplicate word '14' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:47,842 : WARNING : duplicate word 'n' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:47,846 : WARNING : duplicate word 'fr' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:47,850 : WARNING : duplicate word 'e' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:47,853 : WARNING : duplicate word 'samen' in embeddings

2018-10-08 11:34:48,094 : WARNING : duplicate word '' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:48,098 : WARNING : duplicate word 'p' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:48,101 : WARNING : duplicate word 'nit' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:48,103 : WARNING : duplicate word 'econo' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:48,106 : WARNING : duplicate word 'hoofd' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:48,110 : WARNING : duplicate word 'omstandig' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:48,113 : WARNING : duplicate word 'w' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:48,115 : WARNING : duplicate word '' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:48,120 : WARNING : duplicate word 'provincin' i

2018-10-08 11:34:48,415 : WARNING : duplicate word 'open' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:48,416 : WARNING : duplicate word 'edgverdrag' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:48,419 : WARNING : duplicate word 'ae' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:48,423 : WARNING : duplicate word 'orga' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:48,426 : WARNING : duplicate word 'en' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:48,433 : WARNING : duplicate word 'te' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:48,435 : WARNING : duplicate word 'in' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:48,438 : WARNING : duplicate word 'kn' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:48,442 : WARNING : duplicate word 'omzet' i

2018-10-08 11:34:48,631 : WARNING : duplicate word 'ken' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:48,635 : WARNING : duplicate word 'conscintie' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:48,637 : WARNING : duplicate word 'doel' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:48,639 : WARNING : duplicate word 'levens' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:48,641 : WARNING : duplicate word '22' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:48,642 : WARNING : duplicate word 'behande' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:48,649 : WARNING : duplicate word 'elke' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:48,653 : WARNING : duplicate word 'i' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:48,660 : WARNING : duplicate word 

...

2018-10-08 11:34:49,056 : WARNING : duplicate word 'begrotingscommissin' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:49,061 : WARNING : duplicate word 'tre' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:49,061 : WARNING : duplicate word 'argu' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:49,064 : WARNING : duplicate word 'veroor' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:49,066 : WARNING : duplicate word '3' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:49,068 : WARNING : duplicate word 'eze' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:49,072 : WARNING : duplicate word 'par' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:49,076 : WARNING : duplicate word 'l' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:49,081 : WARNING : duplicate wo

2018-10-08 11:34:49,271 : WARNING : duplicate word '27' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:49,272 : WARNING : duplicate word 'deel' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:49,273 : WARNING : duplicate word '00' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:49,275 : WARNING : duplicate word 'dank' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:49,278 : WARNING : duplicate word 'at' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:49,283 : WARNING : duplicate word 'school' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:49,284 : WARNING : duplicate word 'kweek' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:49,288 : WARNING : duplicate word 're' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:49,289 : WARNING : duplicate word 'strijd' i

2018-10-08 11:34:49,463 : WARNING : duplicate word 'va' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:49,466 : WARNING : duplicate word 'verzeke' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:49,469 : WARNING : duplicate word 'wel1' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:49,478 : WARNING : duplicate word 'vee' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:49,481 : WARNING : duplicate word 'as' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:49,484 : WARNING : duplicate word 'afde' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:49,486 : WARNING : duplicate word 'n' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:49,489 : WARNING : duplicate word 'spoe' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:49,491 : WARNING : duplicate word 'pers' in 

2018-10-08 11:34:49,684 : WARNING : duplicate word 'bur' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:49,688 : WARNING : duplicate word 'tus' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:49,690 : WARNING : duplicate word 'nederlandsch-indi' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:49,694 : WARNING : duplicate word 'begro' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:49,698 : WARNING : duplicate word 'wien' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:49,700 : WARNING : duplicate word 'volksvertegenwoor' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:49,700 : WARNING : duplicate word 'on' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:49,703 : WARNING : duplicate word 'raad' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:49,710 : WARNING

2018-10-08 11:34:49,853 : WARNING : duplicate word 'mn' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:49,855 : WARNING : duplicate word 'komen1' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:49,857 : WARNING : duplicate word 'burger' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:49,859 : WARNING : duplicate word 'ten' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:49,865 : WARNING : duplicate word 'o' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:49,871 : WARNING : duplicate word 'sta' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:49,874 : WARNING : duplicate word 'ng' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:49,878 : WARNING : duplicate word '1' in embeddings/vecs_1945-55_v2-2.bin, ignoring all but first
2018-10-08 11:34:49,882 : WARNING : duplicate word 'tijd' in emb

In [35]:
from gensim.models.keyedvectors import KeyedVectors
embeddings_80s = KeyedVectors.load('embeddings/embeddings_1980-1989')
embeddings_80s.most_similar('efficiency', topn=10)

2018-10-03 14:32:37,859 : INFO : loading Word2VecKeyedVectors object from embeddings/embeddings_1980-1989
2018-10-03 14:32:39,069 : INFO : loading vectors from embeddings/embeddings_1980-1989.vectors.npy with mmap=None
2018-10-03 14:32:39,234 : INFO : setting ignored attribute vectors_norm to None
2018-10-03 14:32:39,235 : INFO : loaded embeddings/embeddings_1980-1989
2018-10-03 14:32:39,236 : INFO : precomputing L2-norms of word weight vectors


[('arbeidsbesparende', 0.7315124273300171),
 ('vermogenspositie', 0.7310793995857239),
 ('winstgevendheid', 0.7194138765335083),
 ('marktomstandigheden', 0.7121412754058838),
 ('internationalisatie', 0.7100390195846558),
 ('arbeidskosten', 0.7089903950691223),
 ('bedrijfsvoering', 0.7078852653503418),
 ('produktvernieuwing', 0.7070674300193787),
 ('betalingsverkeer', 0.7054184675216675),
 ('onderwijsuitgaven', 0.7054147720336914)]

In [36]:
import numpy as np
vector = embeddings_80s['efficiency']
zeros = np.zeros_like(vector)
((vector - zeros) ** 2).mean()

0.009667233

In [10]:
import numpy as np
import gensim
from gensim.models.keyedvectors import KeyedVectors
from scipy.spatial.distance import cosine as cosine_distance

In [11]:
def smart_procrustes_align_gensim(base_embed, other_embed, words=None):
	"""Procrustes align two gensim word2vec models (to allow for comparison between same word across models).
	Code ported from HistWords <https://github.com/williamleif/histwords> by William Hamilton <wleif@stanford.edu>.
		(With help from William. Thank you!)
	First, intersect the vocabularies (see `intersection_align_gensim` documentation).
	Then do the alignment on the other_embed model.
	Replace the other_embed model's syn0 and syn0norm numpy matrices with the aligned version.
	Return other_embed.
	If `words` is set, intersect the two models' vocabulary with the vocabulary in words (see `intersection_align_gensim` documentation).
	"""

	# make sure vocabulary and indices are aligned
	in_base_embed, in_other_embed = intersection_align_gensim(base_embed, other_embed, words=words)

	# get the embedding matrices
	base_vecs = in_base_embed.syn0norm
	other_vecs = in_other_embed.syn0norm

	# just a matrix dot product with numpy
	m = other_vecs.T.dot(base_vecs) 
	# SVD method from numpy
	u, _, v = np.linalg.svd(m)
	# another matrix operation
	ortho = u.dot(v) 
	# Replace original array with modified one
	# i.e. multiplying the embedding matrix (syn0norm)by "ortho"
	other_embed.syn0norm = other_embed.syn0 = (other_embed.syn0norm).dot(ortho)
	return other_embed
	
def intersection_align_gensim(m1,m2, words=None):
    """
    Intersect two gensim word2vec models, m1 and m2.
    Only the shared vocabulary between them is kept.
    If 'words' is set (as list or set), then the vocabulary is intersected with this list as well.
    Indices are re-organized from 0..N in order of descending frequency (=sum of counts from both m1 and m2).
    These indices correspond to the new syn0 and syn0norm objects in both gensim models:
        -- so that Row 0 of m1.syn0 will be for the same word as Row 0 of m2.syn0
        -- you can find the index of any word on the .index2word list: model.index2word.index(word) => 2
        The .vocab dictionary is also updated for each model, preserving the count but updating the index.
    """

    # Get the vocab for each model
    vocab_m1 = set(m1.vocab.keys())
    vocab_m2 = set(m2.vocab.keys())

    # Find the common vocabulary
    common_vocab = vocab_m1&vocab_m2
    if words: common_vocab&=set(words)

    # If no alignment necessary because vocab is identical...
    if not vocab_m1-common_vocab and not vocab_m2-common_vocab:
        return (m1,m2)

    # Otherwise sort by frequency (summed for both)
    common_vocab = list(common_vocab)
    common_vocab.sort(key=lambda w: m1.vocab[w].count + m2.vocab[w].count,reverse=True)

    # Then for each model...
    for m in [m1,m2]:
        # Replace old syn0norm array with new one (with common vocab)
        indices = [m.vocab[w].index for w in common_vocab]
        old_arr = m.syn0norm
        new_arr = np.array([old_arr[index] for index in indices])
        m.syn0norm = m.syn0 = new_arr

        # Replace old vocab dictionary with new one (with common vocab)
        # and old index2word with new one
        m.index2word = common_vocab
        old_vocab = m.vocab
        new_vocab = {}
        for new_index,word in enumerate(common_vocab):
            old_vocab_obj=old_vocab[word]
            new_vocab[word] = gensim.models.word2vec.Vocab(index=new_index, count=old_vocab_obj.count)
        m.vocab = new_vocab

    return (m1,m2)

In [50]:
embeddings_t0 = KeyedVectors.load('embeddings/embeddings_1960-1969')
embeddings_t1 = KeyedVectors.load('embeddings/embeddings_1980-1989')

2018-10-03 14:50:37,775 : INFO : loading Word2VecKeyedVectors object from embeddings/embeddings_1960-1969
2018-10-03 14:50:38,968 : INFO : loading vectors from embeddings/embeddings_1960-1969.vectors.npy with mmap=None
2018-10-03 14:50:39,120 : INFO : setting ignored attribute vectors_norm to None
2018-10-03 14:50:39,121 : INFO : loaded embeddings/embeddings_1960-1969
2018-10-03 14:50:39,145 : INFO : loading Word2VecKeyedVectors object from embeddings/embeddings_1980-1989
2018-10-03 14:50:40,661 : INFO : loading vectors from embeddings/embeddings_1980-1989.vectors.npy with mmap=None
2018-10-03 14:50:40,925 : INFO : setting ignored attribute vectors_norm to None
2018-10-03 14:50:40,926 : INFO : loaded embeddings/embeddings_1980-1989


In [54]:
embeddings_t0.most_similar('efficiency', topn=10)

[('rentabiliteit', 0.8244878053665161),
 ('kostenbesparingen', 0.8007539510726929),
 ('specialisatie', 0.7978460788726807),
 ('structurele', 0.7843245267868042),
 ('bedrijfsvoering', 0.7802438139915466),
 ('mechanisatie', 0.7753682136535645),
 ('bedrijfsbeleid', 0.7747997045516968),
 ('premieverhoging', 0.7741789221763611),
 ('textielindustrie', 0.7692541480064392),
 ('verslechtering', 0.7676308155059814)]

In [53]:
embeddings_t1.most_similar('efficiency', topn=10)

[('arbeidsbesparende', 0.7315124273300171),
 ('vermogenspositie', 0.7310793995857239),
 ('winstgevendheid', 0.7194138765335083),
 ('marktomstandigheden', 0.7121412754058838),
 ('internationalisatie', 0.7100390195846558),
 ('arbeidskosten', 0.7089903950691223),
 ('bedrijfsvoering', 0.7078852653503418),
 ('produktvernieuwing', 0.7070674300193787),
 ('betalingsverkeer', 0.7054184675216675),
 ('onderwijsuitgaven', 0.7054147720336914)]

In [55]:
cosine_distance(embeddings_t0['efficiency'], embeddings_t1['efficiency'])

0.9821065347641706

In [56]:
np.linalg.norm(embeddings_t0['efficiency'] - embeddings_t1['efficiency'])

2.5800333

In [57]:
embeddings_t1.similar_by_vector(embeddings_t0['efficiency'] - embeddings_t0['mechanisatie']  + embeddings_t1['efficiency'])

[('efficiency', 0.8065019845962524),
 ('benutting', 0.5967270731925964),
 ('arbeidsbesparende', 0.5935040712356567),
 ('winstgevendheid', 0.577308177947998),
 ('energiebronnen', 0.5744057297706604),
 ('energieverbruik', 0.5736175179481506),
 ('arbeidskosten', 0.5717129707336426),
 ('milieuvriendelijker', 0.5706472992897034),
 ('exporteurs', 0.570343554019928),
 ('produktiemethoden', 0.566421389579773)]

In [58]:
embeddings_t0.syn0.shape

/Users/melvinwevers/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """Entry point for launching an IPython kernel.


(102560, 300)

In [59]:
embeddings_t1.syn0.shape

/Users/melvinwevers/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """Entry point for launching an IPython kernel.


(135267, 300)

In [60]:
embeddings_t1 = smart_procrustes_align_gensim(embeddings_t0, embeddings_t1)

/Users/melvinwevers/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:62: DeprecationWarning: Call to deprecated `syn0norm` (Attribute will be removed in 4.0.0, use self.wv.vectors_norm instead).
/Users/melvinwevers/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:64: DeprecationWarning: Call to deprecated `syn0norm` (Attribute will be removed in 4.0.0, use self.wv.vectors_norm instead).
/Users/melvinwevers/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:64: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
/Users/melvinwevers/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `syn0norm` (Attribute will be removed in 4.0.0, use self.wv.vectors_norm instead).
  app.launch_new_instance()
/Users/melvinwevers/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWa

In [61]:
embeddings_t0.most_similar('efficiency', topn=10)

[('rentabiliteit', 0.8244878053665161),
 ('kostenbesparingen', 0.8007539510726929),
 ('specialisatie', 0.7978460788726807),
 ('structurele', 0.7843245267868042),
 ('bedrijfsvoering', 0.7802438139915466),
 ('mechanisatie', 0.7753682136535645),
 ('premieverhoging', 0.7741789221763611),
 ('textielindustrie', 0.7692541480064392),
 ('verslechtering', 0.7676308155059814),
 ('besparingen', 0.7595827579498291)]

In [62]:
embeddings_t1.most_similar('efficiency', topn=10)

[('arbeidsbesparende', 0.7315124273300171),
 ('vermogenspositie', 0.7310794591903687),
 ('internationalisatie', 0.7100391387939453),
 ('arbeidskosten', 0.7089903950691223),
 ('bedrijfsvoering', 0.7078853249549866),
 ('betalingsverkeer', 0.7054184675216675),
 ('energieverbruik', 0.703171968460083),
 ('overheidsinvesteringen', 0.6992776393890381),
 ('belastingfaciliteiten', 0.6974428296089172),
 ('renderende', 0.6950628757476807)]

In [63]:
cosine_distance(embeddings_t0['efficiency'], embeddings_t1['efficiency'])

0.3025396466255188

In [64]:
np.linalg.norm(embeddings_t0['efficiency'] - embeddings_t1['efficiency'])

0.77786833

In [65]:
embeddings_t1.similar_by_vector(embeddings_t0['efficiency'] - embeddings_t0['mechanisatie']  + embeddings_t1['efficiency'])

[('efficiency', 0.8239121437072754),
 ('vermogenspositie', 0.6163286566734314),
 ('bedrijfsvoering', 0.6134113669395447),
 ('tijdige', 0.6103959083557129),
 ('prestatiebeloning', 0.6022935509681702),
 ('betalingsverkeer', 0.5989818572998047),
 ('verzwaring', 0.5975215435028076),
 ('arbeidsomstandigheden', 0.5889613032341003),
 ('pensioenregeling', 0.5887690782546997),
 ('opleidingsmogelijkheden', 0.5872750282287598)]

In [14]:
for t0_path, t1_path in [('embeddings/vecs_1945-55.bin', 'embeddings/vecs_1965-75.bin')]:
    embeddings_t0 = KeyedVectors.load_word2vec_format(t0_path, binary=True, unicode_errors='ignore')
    embeddings_t1 = KeyedVectors.load_word2vec_format(t1_path, binary=True, unicode_errors='ignore')
    embeddings_t0.init_sims(replace=True)
    embeddings_t1.init_sims(replace=True)
    embeddings_t1 = smart_procrustes_align_gensim(embeddings_t0, embeddings_t1)
    embeddings_t0.save('{0}_aligned'.format(t0_path))
    embeddings_t1.save('{0}_aligned'.format(t1_path))

2018-10-04 10:33:27,077 : INFO : loading projection weights from embeddings/vecs_1945-55.bin
2018-10-04 10:33:27,106 : WARNING : duplicate word 'n' in embeddings/vecs_1945-55.bin, ignoring all but first
2018-10-04 10:33:27,123 : WARNING : duplicate word 'n' in embeddings/vecs_1945-55.bin, ignoring all but first
2018-10-04 10:33:27,142 : WARNING : duplicate word 'f' in embeddings/vecs_1945-55.bin, ignoring all but first
2018-10-04 10:33:27,148 : WARNING : duplicate word '2' in embeddings/vecs_1945-55.bin, ignoring all but first
2018-10-04 10:33:27,155 : WARNING : duplicate word '—' in embeddings/vecs_1945-55.bin, ignoring all but first
2018-10-04 10:33:27,159 : WARNING : duplicate word '' in embeddings/vecs_1945-55.bin, ignoring all but first
2018-10-04 10:33:27,168 : WARNING : duplicate word '3' in embeddings/vecs_1945-55.bin, ignoring all but first
2018-10-04 10:33:27,182 : WARNING : duplicate word '4' in embeddings/vecs_1945-55.bin, ignoring all but first
2018-10-04 10:33:27,184 : WA

2018-10-04 10:33:27,568 : WARNING : duplicate word 'staats' in embeddings/vecs_1945-55.bin, ignoring all but first
2018-10-04 10:33:27,572 : WARNING : duplicate word 'her' in embeddings/vecs_1945-55.bin, ignoring all but first
2018-10-04 10:33:27,582 : WARNING : duplicate word 'inter' in embeddings/vecs_1945-55.bin, ignoring all but first
2018-10-04 10:33:27,587 : WARNING : duplicate word 'mi' in embeddings/vecs_1945-55.bin, ignoring all but first
2018-10-04 10:33:27,591 : WARNING : duplicate word '15' in embeddings/vecs_1945-55.bin, ignoring all but first
2018-10-04 10:33:27,596 : WARNING : duplicate word 'mede' in embeddings/vecs_1945-55.bin, ignoring all but first
2018-10-04 10:33:27,598 : WARNING : duplicate word 're' in embeddings/vecs_1945-55.bin, ignoring all but first
2018-10-04 10:33:27,605 : WARNING : duplicate word 'wan' in embeddings/vecs_1945-55.bin, ignoring all but first
2018-10-04 10:33:27,609 : WARNING : duplicate word 'moge' in embeddings/vecs_1945-55.bin, ignoring al

2018-10-04 10:33:27,935 : WARNING : duplicate word 'p' in embeddings/vecs_1945-55.bin, ignoring all but first
2018-10-04 10:33:27,944 : WARNING : duplicate word 'ij' in embeddings/vecs_1945-55.bin, ignoring all but first
2018-10-04 10:33:27,948 : WARNING : duplicate word 'indo' in embeddings/vecs_1945-55.bin, ignoring all but first
2018-10-04 10:33:27,953 : WARNING : duplicate word 'amen' in embeddings/vecs_1945-55.bin, ignoring all but first
2018-10-04 10:33:27,956 : WARNING : duplicate word 'poli' in embeddings/vecs_1945-55.bin, ignoring all but first
2018-10-04 10:33:27,957 : WARNING : duplicate word 'ber' in embeddings/vecs_1945-55.bin, ignoring all but first
2018-10-04 10:33:27,959 : WARNING : duplicate word 'belasting' in embeddings/vecs_1945-55.bin, ignoring all but first
2018-10-04 10:33:27,963 : WARNING : duplicate word 'betrek' in embeddings/vecs_1945-55.bin, ignoring all but first
2018-10-04 10:33:27,965 : WARNING : duplicate word 'uitge' in embeddings/vecs_1945-55.bin, igno

2018-10-04 10:33:28,339 : WARNING : duplicate word 'boven' in embeddings/vecs_1945-55.bin, ignoring all but first
2018-10-04 10:33:28,340 : WARNING : duplicate word 'o' in embeddings/vecs_1945-55.bin, ignoring all but first
2018-10-04 10:33:28,344 : WARNING : duplicate word 'n' in embeddings/vecs_1945-55.bin, ignoring all but first
2018-10-04 10:33:28,346 : WARNING : duplicate word 'omzet' in embeddings/vecs_1945-55.bin, ignoring all but first
2018-10-04 10:33:28,347 : WARNING : duplicate word 'aan' in embeddings/vecs_1945-55.bin, ignoring all but first
2018-10-04 10:33:28,348 : WARNING : duplicate word 'arti' in embeddings/vecs_1945-55.bin, ignoring all but first
2018-10-04 10:33:28,351 : WARNING : duplicate word 'beschou' in embeddings/vecs_1945-55.bin, ignoring all but first
2018-10-04 10:33:28,361 : WARNING : duplicate word 'wel' in embeddings/vecs_1945-55.bin, ignoring all but first
2018-10-04 10:33:28,362 : WARNING : duplicate word 'neer' in embeddings/vecs_1945-55.bin, ignoring 

2018-10-04 10:33:28,600 : WARNING : duplicate word 'beant' in embeddings/vecs_1945-55.bin, ignoring all but first
2018-10-04 10:33:28,601 : WARNING : duplicate word 'pre' in embeddings/vecs_1945-55.bin, ignoring all but first
2018-10-04 10:33:28,602 : WARNING : duplicate word 'enqutecommissie' in embeddings/vecs_1945-55.bin, ignoring all but first
2018-10-04 10:33:28,602 : WARNING : duplicate word 'pen' in embeddings/vecs_1945-55.bin, ignoring all but first
2018-10-04 10:33:28,603 : WARNING : duplicate word 'depar' in embeddings/vecs_1945-55.bin, ignoring all but first
2018-10-04 10:33:28,604 : WARNING : duplicate word 'kamer' in embeddings/vecs_1945-55.bin, ignoring all but first
2018-10-04 10:33:28,606 : WARNING : duplicate word '1berichten' in embeddings/vecs_1945-55.bin, ignoring all but first
2018-10-04 10:33:28,608 : WARNING : duplicate word 'weer' in embeddings/vecs_1945-55.bin, ignoring all but first
2018-10-04 10:33:28,610 : WARNING : duplicate word 'reke' in embeddings/vecs_1

...

2018-10-04 10:33:31,243 : WARNING : duplicate word 'lom' in embeddings/vecs_1965-75.bin, ignoring all but first
2018-10-04 10:33:31,251 : WARNING : duplicate word 'nood' in embeddings/vecs_1965-75.bin, ignoring all but first
2018-10-04 10:33:31,257 : WARNING : duplicate word 'wer' in embeddings/vecs_1965-75.bin, ignoring all but first
2018-10-04 10:33:31,259 : WARNING : duplicate word 'staats' in embeddings/vecs_1965-75.bin, ignoring all but first
2018-10-04 10:33:31,267 : WARNING : duplicate word 'communiqu' in embeddings/vecs_1965-75.bin, ignoring all but first
2018-10-04 10:33:31,278 : WARNING : duplicate word 'mo' in embeddings/vecs_1965-75.bin, ignoring all but first
2018-10-04 10:33:31,282 : WARNING : duplicate word 'tom' in embeddings/vecs_1965-75.bin, ignoring all but first
2018-10-04 10:33:31,289 : WARNING : duplicate word 'belas' in embeddings/vecs_1965-75.bin, ignoring all but first
2018-10-04 10:33:31,290 : WARNING : duplicate word '20' in embeddings/vecs_1965-75.bin, ignor

2018-10-04 10:33:31,568 : WARNING : duplicate word 'at' in embeddings/vecs_1965-75.bin, ignoring all but first
2018-10-04 10:33:31,569 : WARNING : duplicate word 'continu' in embeddings/vecs_1965-75.bin, ignoring all but first
2018-10-04 10:33:31,576 : WARNING : duplicate word 'rebro' in embeddings/vecs_1965-75.bin, ignoring all but first
2018-10-04 10:33:31,578 : WARNING : duplicate word 'wt' in embeddings/vecs_1965-75.bin, ignoring all but first
2018-10-04 10:33:31,588 : WARNING : duplicate word 'per' in embeddings/vecs_1965-75.bin, ignoring all but first
2018-10-04 10:33:31,590 : WARNING : duplicate word 'poli' in embeddings/vecs_1965-75.bin, ignoring all but first
2018-10-04 10:33:31,594 : WARNING : duplicate word 'al' in embeddings/vecs_1965-75.bin, ignoring all but first
2018-10-04 10:33:31,597 : WARNING : duplicate word 'mest' in embeddings/vecs_1965-75.bin, ignoring all but first
2018-10-04 10:33:31,599 : WARNING : duplicate word 'vr' in embeddings/vecs_1965-75.bin, ignoring al

2018-10-04 10:33:31,918 : WARNING : duplicate word 'ontwik' in embeddings/vecs_1965-75.bin, ignoring all but first
2018-10-04 10:33:31,924 : WARNING : duplicate word 'beant' in embeddings/vecs_1965-75.bin, ignoring all but first
2018-10-04 10:33:31,926 : WARNING : duplicate word 'belasting' in embeddings/vecs_1965-75.bin, ignoring all but first
2018-10-04 10:33:31,931 : WARNING : duplicate word 'di' in embeddings/vecs_1965-75.bin, ignoring all but first
2018-10-04 10:33:31,934 : WARNING : duplicate word 'en' in embeddings/vecs_1965-75.bin, ignoring all but first
2018-10-04 10:33:31,938 : WARNING : duplicate word 'begro' in embeddings/vecs_1965-75.bin, ignoring all but first
2018-10-04 10:33:31,939 : WARNING : duplicate word 'zr' in embeddings/vecs_1965-75.bin, ignoring all but first
2018-10-04 10:33:31,946 : WARNING : duplicate word 'vraag' in embeddings/vecs_1965-75.bin, ignoring all but first
2018-10-04 10:33:31,947 : WARNING : duplicate word 'werkgelegen' in embeddings/vecs_1965-75.

2018-10-04 10:33:32,504 : WARNING : duplicate word 'doel' in embeddings/vecs_1965-75.bin, ignoring all but first
2018-10-04 10:33:32,510 : WARNING : duplicate word 'moeilijk' in embeddings/vecs_1965-75.bin, ignoring all but first
2018-10-04 10:33:32,518 : WARNING : duplicate word 'pre' in embeddings/vecs_1965-75.bin, ignoring all but first
2018-10-04 10:33:32,520 : WARNING : duplicate word 'wetenschap' in embeddings/vecs_1965-75.bin, ignoring all but first
2018-10-04 10:33:32,522 : WARNING : duplicate word 'hou' in embeddings/vecs_1965-75.bin, ignoring all but first
2018-10-04 10:33:32,525 : WARNING : duplicate word 'duits' in embeddings/vecs_1965-75.bin, ignoring all but first
2018-10-04 10:33:32,527 : WARNING : duplicate word 'bouw' in embeddings/vecs_1965-75.bin, ignoring all but first
2018-10-04 10:33:32,535 : WARNING : duplicate word 'indi' in embeddings/vecs_1965-75.bin, ignoring all but first
2018-10-04 10:33:32,536 : WARNING : duplicate word '2' in embeddings/vecs_1965-75.bin, 

2018-10-04 10:33:34,919 : INFO : not storing attribute vectors_norm
2018-10-04 10:33:35,283 : INFO : saved embeddings/vecs_1945-55.bin_aligned
2018-10-04 10:33:35,285 : INFO : saving Word2VecKeyedVectors object under embeddings/vecs_1965-75.bin_aligned, separately None
2018-10-04 10:33:35,287 : INFO : not storing attribute vectors_norm
2018-10-04 10:33:35,594 : INFO : saved embeddings/vecs_1965-75.bin_aligned
